###Need:
####Is there a pattern to monthly variations in the ratings of businesses? Comparing reviews to the average for that business, or each month to the prior month, do users rate businesses higher or lower than the average in certain months? Is this more of a seasonal or monthly issue? Does this vary geographically or by other sub-category?

###Introduction:
Yelp connects people with great local businesses by providing a platform for businesses and consumers to engage and transact. The contributions to Yelp's platform is in the form of reviews and ratings, tips, photos and videos. The contributions that matter to businesses with respect to our question would be reviews and ratings. In order to find out if a pattern existed in the ratings received each month, the datasets provided as part of this project had to be explored. To answer our question we used datasets 'review', 'business' and 'categories'. They were downloaded from https://www.yelp.com/dataset to the yelpdata folder within the databricks file system.

### The Initial Approach:
To better understand what 'Comparing reviews to the average for that business' meant, we searched for an example in Yelp that would give us an idea about how users rate businesses. An example from prettyReviewFirst100.txt within the https://sjsu.instructure.com/courses/1263010/files/50919706 was considered. The example had been rated by the user as '2' within the txt file. On trying to filter the business_id within the business dataset file, it was found that the average rating given was '4' which agrees with the page on Yelp but was rounded to half star increments. In order to have a granular calculation of the average, the average had to be calculated for the review ids for a business id. For the example considered, we found that the average calculated was obtained as '4.13'. Taking the difference, we compared this calculated average to the single review considered intially. This would tell if the rating provided by a user for that business stood above or below average. Further the same had to be done for all reviews across months to find out if a pattern existed. 

The data wrangling for this approach is provided below.

###Reading the review dataset 
To work towards answering the question, the review dataset was first read from the yelpdata folder which was initially uploaded from https://www.yelp.com/dataset.

In [5]:
df_review = spark.read.json('/yelpdata/review.bz2')
print("number of reviews:",df_review.count())
df_review.printSchema()

number of reviews: 5996996
root
-- business_id: string (nullable = true)
-- cool: long (nullable = true)
-- date: string (nullable = true)
-- funny: long (nullable = true)
-- review_id: string (nullable = true)
-- stars: long (nullable = true)
-- text: string (nullable = true)
-- useful: long (nullable = true)
-- user_id: string (nullable = true)

In this notebook, most of the joining has been done by using views (created from the respective dataframes).  This allows the sql queries to be used directly.

In [7]:
# The view created 'view_OfReviews' is named because it is a view created from the review dataframe
df_review.createOrReplaceTempView("view_OfReviews")

##### We now take relevant fields from the review dataframe and extract month and year from the date field since the question asks us to compare it across time. Fields needed to answer this question with respect to the review dataset are business_id, stars, month and year.

In [9]:
df_review_selected = spark.sql("""
SELECT business_id, review_id, stars, MONTH(date) AS month, YEAR(date) AS year
FROM view_OfReviews
ORDER BY year, month
""")
print("number of reviews:",df_review_selected.count())
df_review_selected.show(truncate=False)

number of reviews: 5996996
+----------------------+----------------------+-----+-----+----+
business_id |review_id |stars|month|year|
+----------------------+----------------------+-----+-----+----+
GCRvrxMSC1nzShyM4Y-guQ|0O7kc-T1SrZpF84jMOyeOg|5 |10 |2004|
u8C8pRvaHXg3PgDrsUHJHQ|0QHCY_55TFHHvyumEMpDew|4 |10 |2004|
na4Th5DrNauOv-c43QQFvA|xW294l3Lwh0cxlHU1jwRDA|5 |10 |2004|
EZOoB2D8uQHV_gJoGCMTxQ|1Iobyi_7BkFON25Oegs0aw|4 |10 |2004|
PD2MAlYYi9HCqPH7IBKwTg|_CRpX4FGBkxie_1q0-DbjQ|5 |10 |2004|
oYMsq2Xvzw6UbrIlMWjb-A|2F5J51OYtD49eyIUKJKVgg|4 |10 |2004|
AtLv64FV-Pw6JuT3XUKU1g|pho1XNCTeRxQVzWR_5vacg|4 |10 |2004|
2Q2uE1dwhqMFOE5CYfvMEQ|DrP3TrvOtE_DKlAPaEVYJQ|5 |10 |2004|
u8C8pRvaHXg3PgDrsUHJHQ|JDBubAcRw4FXfg1c5xk-dA|5 |10 |2004|
ydUqgWsF3F27TbauOyib0w|Ef1skKLKZ9izwBmreb_-qw|4 |12 |2004|
ikubvyZFO0kxhA56RETzIg|mNNTXbRPA6xKsWAFEJekdA|5 |12 |2004|
N2PlDjUJVfOJzsPzY0Au1w|6POnAs_4MijROSKeOevXHQ|3 |12 |2004|
iY_N3YhWL2_CUnBy41WK3w|haQe4wzpm3eCfUk79uqjBg|5 |12 |2004|
ii8sAGBexBOJoYRFafF9XQ|VwlljS4oxU4VyP2ODo9gwA|5 |1 |2005|
ZkGDCVKSdf8m76cnnalL-A|gfPLT7BTqd2mJQ449BTrpw|4 |1 |2005|
LJvGjabschy4zGXwnAk2Wg|pAugo1esC40DFFsXzcxpQA|4 |1 |2005|
na4Th5DrNauOv-c43QQFvA|m6vR3SHcSi9qfgmQDIGE5Q|5 |1 |2005|
ZkGDCVKSdf8m76cnnalL-A|Z54-ssb69Mx_lG1YtJ6N1g|5 |1 |2005|
qjnJFZtsY_nfRzoL3J_UWQ|NkAH3of2_x-3pTng3r1jsw|5 |1 |2005|
LkMtMHVetws5_7QfRjPtlg|Dm7Jh7tVp_97sOxEYH4DnA|4 |1 |2005|
+----------------------+----------------------+-----+-----+----+
only showing top 20 rows

In [10]:
#The view created is named 'view_OfReviews_selected' as it is obtained by selecting only the relevant fields from 'df_review_selected' 
df_review_selected.createOrReplaceTempView("view_OfReviews_selected")

#####To have a granular calculation of the average of stars, we take the review data and group it by business_id and then calculate the average stars for that business_id

In [12]:
df_review_avgstars = spark.sql("""
SELECT business_id, COUNT(review_id) AS reviewcount, AVG(stars) AS avg_stars
FROM view_OfReviews_selected
GROUP BY business_id
""")
print("number of records:",df_review_avgstars.count())
df_review_avgstars.show(truncate=False)

number of records: 188593
+----------------------+-----------+------------------+
business_id |reviewcount|avg_stars |
+----------------------+-----------+------------------+
--9e1ONYQuAa-CB_Rrw7Tw|1546 |4.102846054333765 |
j2bx_ctb_ED3zbfSaqhQmQ|1001 |4.213786213786213 |
v6n4I3fA8eqmO4bnKieOIw|44 |2.977272727272727 |
5mLnpk7lJuZ8baMGZhwa5g|13 |2.3076923076923075|
RsREl3LkFGiwbt3kGIX4Eg|296 |2.814189189189189 |
d2fkRF67jiASrXxHfbmJuA|3 |2.6666666666666665|
G58YATMKnn-M-RUDWg3lxw|119 |3.546218487394958 |
KGYHQyb83WDg5YsPtPZkwg|21 |3.9523809523809526|
zgGoxOsThXKAcs2lAgC9yg|351 |3.1253561253561255|
CAmPPSKoFCd8zo2bxRxRkA|39 |3.7948717948717947|
rWtglX0leY_pqi-UD820Ag|3 |5.0 |
2UgRg5a6KmpbD_SZfhNrKg|392 |4.326530612244898 |
35X1ZV9tSEqB__yJEAhuhQ|455 |2.9934065934065934|
JFhfVTt_8xNjtwqSD-raIw|125 |3.096 |
Zo3fwmOUaDpKvUhLNp9erA|49 |3.6530612244897958|
rNKaormrHcdJhA_HDmUe6A|98 |3.4591836734693877|
FpFIAW_IEvASZBbusABbkA|63 |3.5714285714285716|
Yb_vynxYrpw-QM3cXPx0hQ|5 |4.0 |
BD18SKv935HDmlKrLPkhLA|18 |4.555555555555555 |
L7iOEZLuvd3vHoX8Efi_hw|27 |2.3703703703703702|
+----------------------+-----------+------------------+
only showing top 20 rows

#####From the above result, if we consider the business that received 1546 reviews, the average stars obtained is 4.102 and if we consider a business that received only 3 reviews, the average star that they received is 5 stars. At this point, we don't think that taking the average rating of a business and comparing it to other business to find if a user rated a business higher or lower to the others, would be a correct approach, since there are businesses that received really low reviews which could be due to various other reasons. Yet, in order to find out more about the data and how it could relate to the question, we have continued with our data analysis. Moving down further in our analysis, we are making a decision to limit our analysis only to data that have review count more than 10 since we find that there are so many businesses that have received reviews less than 10.

In [14]:
#'view_OfReviews_avgstars' is named since it contains the calculated average of the ratings of the review_ids grouped by the related business_ids  
df_review_avgstars.createOrReplaceTempView("view_OfReviews_avgstars")

Using 'view_OfReviews' (which contains the selected fields from the review dataset and the extracted month and year fields) and 'view_OfReviews_avgstars' (where we took the average of each businesses), we create a join which allows us to compare each review in a business to the average received by the same business and observe the variance.

In [16]:
df_review_compare_toavgOfbusiness_original = spark.sql("""
SELECT vrs.business_id, vrs.review_id, vrs.stars, ROUND(vra.avg_stars,4) AS avgstars, ROUND((vrs.stars-vra.avg_stars),4) AS diff_avgstars, vrs.month, vrs.year
FROM view_OfReviews_selected vrs JOIN view_OfReviews_avgstars vra ON vrs.business_id = vra.business_id
""")
print("no. of records:",df_review_compare_toavgOfbusiness_original.count())
df_review_compare_toavgOfbusiness_original.show()

no. of records: 5996996
+--------------------+--------------------+-----+--------+-------------+-----+----+
 business_id| review_id|stars|avgstars|diff_avgstars|month|year|
+--------------------+--------------------+-----+--------+-------------+-----+----+
--9e1ONYQuAa-CB_R...|lgpSS6UsKYIvnQaw8...| 5| 4.1028| 0.8972| 4|2005|
--9e1ONYQuAa-CB_R...|B45SjrSgVXDw2EPlF...| 5| 4.1028| 0.8972| 1|2006|
--9e1ONYQuAa-CB_R...|rIwd3sRWVrN7d0vxd...| 5| 4.1028| 0.8972| 1|2006|
--9e1ONYQuAa-CB_R...|aQQIcQfFYawLYU85o...| 5| 4.1028| 0.8972| 3|2006|
--9e1ONYQuAa-CB_R...|-7O72HqM4eJF0NMCg...| 5| 4.1028| 0.8972| 3|2006|
--9e1ONYQuAa-CB_R...|Y9mls1kWNRsHYURCt...| 4| 4.1028| -0.1028| 5|2006|
--9e1ONYQuAa-CB_R...|d3-sC4eUvIdDzz6Kg...| 4| 4.1028| -0.1028| 7|2006|
--9e1ONYQuAa-CB_R...|y3EBG-1zykvZblLdS...| 4| 4.1028| -0.1028| 7|2006|
--9e1ONYQuAa-CB_R...|tCpIO0BO1f916-Ch9...| 5| 4.1028| 0.8972| 9|2006|
--9e1ONYQuAa-CB_R...|bHMxqjTDchcYo8pqw...| 4| 4.1028| -0.1028| 9|2006|
--9e1ONYQuAa-CB_R...|8Lkdb_TyJDn8knJX0...| 4| 4.1028| -0.1028| 9|2006|
--9e1ONYQuAa-CB_R...|pg2KJCg1stKrexSzg...| 5| 4.1028| 0.8972| 11|2006|
--9e1ONYQuAa-CB_R...|He1MijYelqA7uZxH0...| 5| 4.1028| 0.8972| 12|2006|
--9e1ONYQuAa-CB_R...|WtIWqoHjAruu__QYt...| 4| 4.1028| -0.1028| 1|2007|
--9e1ONYQuAa-CB_R...|tYIxOLwcZfnC5Wrht...| 3| 4.1028| -1.1028| 2|2007|
--9e1ONYQuAa-CB_R...|IsFiM2sCdnkYnQN1K...| 5| 4.1028| 0.8972| 3|2007|
--9e1ONYQuAa-CB_R...|miW64R3yPbSWwCJtj...| 4| 4.1028| -0.1028| 3|2007|
--9e1ONYQuAa-CB_R...|z6RWTqn5dZYPSQxx1...| 3| 4.1028| -1.1028| 4|2007|
--9e1ONYQuAa-CB_R...|tZIL8bm90mjE7zgtd...| 5| 4.1028| 0.8972| 4|2007|
--9e1ONYQuAa-CB_R...|hwmEJHtumKQOHqe85...| 3| 4.1028| -1.1028| 4|2007|
+--------------------+--------------------+-----+--------+-------------+-----+----+
only showing top 20 rows

The view in the above step, 'view_review_compare_toavgOfbusiness_original' was created so that it could be joined to the business dataset, to further answer the rest of the question

In [18]:
#'view_review_compare_toavgOfbusiness_original' is named since it contained the comparison data between the stars and the average of stars related to each business from the review dataset. This is, this is the result that is obtained prior to filtering the businesses with low review count.
df_review_compare_toavgOfbusiness_original.createOrReplaceTempView("view_review_compare_toavgOfbusiness_original")


#####To download the dataset to further use for Data visualization, we would need to use the below command which would help to first create a csv file. We are writing our data out to the /FileStore/tables/ directory because we can download from there. The file name specified is review_compare_toavgOfbusiness_original. Looking at the tables folder allows us to find a folder similar to that of the filename.

In [20]:
#df_review_compare_toavgOfbusiness_original.repartition(1).write.option("header","true").option("sep","\t").option("mode","overwrite").csv("/FileStore/tables/review_compare_toavgOfbusiness_original")

#####We now move the csv file and store it under the tables folder as a '.tsv' file

In [22]:
#dbutils.fs.mv("/FileStore/tables/review_compare_toavgOfbusiness_original/part-00000-tid-1504980756658415970-4439d3c1-3f01-4c7f-a932-ff2d8371554c-3861-c000.csv", "/FileStore/tables/review_compare_toavgOfbusiness_original.tsv")

#####Below, the folder (which is created with the filename) is removed

In [24]:
#dbutils.fs.rm("/FileStore/tables/review_compare_toavgOfbusiness_original", recurse=True)

#####Using the below command, the '.tsv' file could be downloaded from the path mentioned and then could be used to import into tableau.

In [26]:
#https://community.cloud.databricks.com/files/tables/review_compare_toavgOfbusiness_original.tsv?o=5722899434425902#notebook/

#####We're using the code below since Databricks is not converting the images automatically to the Base64 representations that would be visible in published notebooks.  The cell below contains the code with the function that would generate the Base64 representation for the images imported further in the notebook. This function will be useful later on. Also there is then another cell that has the a line of code which we would need to use to display the images in the notebook.

In [28]:
import base64
from PIL import Image

def showimage(path):
  image_string = ""
  width = height = 0
  # Get the base64 string for the image
  with open(path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
  # Get the width and height of the image in pixels
  with Image.open(path) as img:
    width, height = img.size
  # Build the image tag
  img_tag = '''
  <style>
  div {
    min-width: %ipx;
    max-width: %ipx;
  }
  </style>
  <div><img src="data:image/png;base64, %s"  style="width:%ipx;" /></div>''' % (width,width,image_string, width)
  return(img_tag)

#####The graph below shows how the users have rated businesses over the months. We don't see variation but they definitely seem to have been rated on the higher side more than the lower (seeing as the median is higher than the 0).

In [30]:
displayHTML( showimage("/dbfs/FileStore/tables/ComparingReviewsToAvgOfBusiness.PNG") )

#####From this point onward, the data wrangling will be performed on filtered records (reviews > 10), which was mentioned earlier in command 13

#####We now filter 'df_review_compare_toavgOfbusiness_original', to only show records with review count above 10 . The result filters  539,121 records from the original dataframe. This is quite a large number and fairly a major decision (to omit this data) as we move on with our wrangling. 
A point of note at this point is that we are not sure how it would impact our data analysis process.

In [33]:
df_review_compare_toavgOfbusiness = spark.sql("""
SELECT vrs.business_id, vra.reviewcount, vrs.stars, ROUND(vra.avg_stars,4) AS avgstars, ROUND((vrs.stars-vra.avg_stars),4) AS diff_avgstars, vrs.month, vrs.year
FROM view_OfReviews_selected vrs JOIN view_OfReviews_avgstars vra ON vrs.business_id = vra.business_id
WHERE vra.reviewcount > 10
""")
print("no. of records:",df_review_compare_toavgOfbusiness.count())
df_review_compare_toavgOfbusiness.show()

no. of records: 5457875
+--------------------+-----------+-----+--------+-------------+-----+----+
 business_id|reviewcount|stars|avgstars|diff_avgstars|month|year|
+--------------------+-----------+-----+--------+-------------+-----+----+
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 4|2005|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 1|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 1|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 5|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 7|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 7|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 9|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 9|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 9|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 11|2006|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 12|2006|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 1|2007|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 2|2007|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2007|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 3|2007|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 4|2007|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 4|2007|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 4|2007|
+--------------------+-----------+-----+--------+-------------+-----+----+
only showing top 20 rows

#####Although we have used the filter to keep only those businesses with review count greater than 10, yet we do not see much of a difference from the above graph where all the businesses were included.

In [35]:
displayHTML( showimage("/dbfs/FileStore/tables/ComparingReviewsToAvgOfBusinessesGreater10.png") )

##### Graphs for 2004 and 2005: whereas the intiial year had hardly any data, 2005 reflects variations (random) over months

In [37]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBusiness_2004.png") )

In [38]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBusin_2005.png") )

##### 2006 and onwards to 2017, any pattern that was observed in the earlier years, is not seen anymore. Even though users still rate businesses more high than low as compared to the average, there is no pattern to it over the months

In [40]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBusi_2006.png") )

In [41]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBuss_2017.png") )

##### 2018 follows the trend of recent years; users do rate slightly higher than average but no real trend in any month

In [43]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBusin_2018.png") )

##### The median values for every year is higher than the average rating of the category

In [45]:
displayHTML( showimage("/dbfs/FileStore/tables/Filter10_CompAvgBuss_Year.png") )

#####To answer our question 'Comparing reviews to the average for business across months and to find out if users rate businesses higher or lower than the average in certain months' based on category or geographically, we did the following steps

The view, 'view_review_compare_toavgOfbusiness' was created to be used at a later step (to join with the business and category datasets), which would answer the last part of the question.

In [48]:
# The view is named as 'view_review_compare_toavgOfbusiness' as it is extracted from the previously obtained dataframe 'df_review_compare_toavgOfbusiness_original'
df_review_compare_toavgOfbusiness.createOrReplaceTempView("view_review_compare_toavgOfbusiness")

####Adding on, we could also find out if that pattern holds by category. For this, first, we would need to bring in the business dataset. The business dataset is downloaded from the Yelp dataset webpage https://www.yelp.com/dataset, into the yelpdata folder within the DBFS. The dataset is then loaded into the notebook using the below steps.

In [50]:
df_business = spark.read.json('/yelpdata/business.bz2')
print( "number of businesses:", df_business.count())
df_business.printSchema()

number of businesses: 188593
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- neighborhood: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

The below view is created for the business dataframe so that sql queries could be used directly for data wrangling purpose.

In [52]:
#creating a view for the fields from the business dataset
df_business.createOrReplaceTempView("view_OfBusiness")

#####Selecting relevant fields from the business dataset to answer our question. This includes business_id (that could be used to join records between the business dataset) and the review dataset.

In [54]:
df_business_selected=spark.sql("""
SELECT business_id, categories, state
FROM view_OfBusiness
""")
print( "number of businesses:", df_business_selected.count())
df_business_selected.show(truncate=False)

number of businesses: 188593
+----------------------+---------------------------------------------------------------------------------------------------------------------+-----+
business_id |categories |state|
+----------------------+---------------------------------------------------------------------------------------------------------------------+-----+
Apn5Q_b6Nz61Tq4XzPdf9A|Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel |AB |
AjEbIBw6ZFfln7ePHha9PA|Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services|NV |
O8S5hYJ1SMc8fA4QBtVujA|Breakfast & Brunch, Restaurants, French, Sandwiches, Cafes |QC |
bFzdJJ3wp3PZssNEsyU23g|Insurance, Financial Services |AZ |
8USyCYqpScwiNEb58Bt6CA|Home & Garden, Nurseries & Gardening, Shopping, Local Services, Automotive, Electronics Repair |AB |
45bWSZtniwPRiqlivpS8Og|Coffee & Tea, Food |AZ |
9A2quhZLyWk0akUetBd8hQ|Food, Bakeries |ON |
6OuOZAok8ikONMS_T3EzXg|Restaurants, Thai |ON |
8-NRKkPY1UiFXW20WXKiXg|Mexican, Restaurants |AZ |
UTm5QZThPQlT35mkAcGOjg|Flowers & Gifts, Gift Shops, Shopping |PA |
KapTdGyGs7RK0c68Z6hhhg|Restaurants, Japanese |ON |
tZnSodhPwNr4bzrwJ1CSbw|Cajun/Creole, Southern, Restaurants |ON |
_c3ixq9jYKxhLUB0czi0ug|Bars, Sports Bars, Dive Bars, Burgers, Nightlife, Sandwiches, Restaurants |AZ |
bBUMib8l6Me1ZB1_Qkezkg|Restaurants, Pakistani, Indian, Middle Eastern |QC |
o_zDURWzgt5yM5HW_xGJsg|Beauty & Spas, Barbers |AB |
IbTDLI02Sq50C7B-EWK1sg|Delis, Restaurants, Sandwiches |AB |
r89Re4FNgVWHgBfjCVZyVw|Nightlife, Bars, American (Traditional), Tapas/Small Plates, Poutineries, Supper Clubs, Restaurants, Tapas Bars |QC |
hTzcHtk4-0QJnFUbkKpd5Q|Shopping, Fashion, Department Stores |NV |
UwIpS9UKsPiKAv1fiEYhqg|Financial Services, Check Cashing/Pay-day Loans, Title Loans |NV |
vJIuDBdu01vCA8y1fwR1OQ|American (Traditional), Food, Bakeries, Restaurants |NV |
+----------------------+---------------------------------------------------------------------------------------------------------------------+-----+
only showing top 20 rows

In [55]:
#creating a view for the selected fields from the business dataset
df_business_selected.createOrReplaceTempView("view_OfBusiness_selected")

#### We would also want to bring in the top-level categories from the categories dataset and match up to the categories in the business dataset. For this, first we have to load the categories dataset, which is a json file (downloaded from https://www.yelp.com/dataset and then imported into the 'yelp folder'within the DBFS)

In [57]:
df_categories = spark.read.option("multiline","true").json("/yelp/categories.json")
print( "number of categories:", df_categories.count())
df_categories.show()
df_categories.printSchema()

number of categories: 1539
+------------------+--------------------+-----------------+---------------+-------------------+
 alias| country_blacklist|country_whitelist| parents| title|
+------------------+--------------------+-----------------+---------------+-------------------+
 3dprinting| null| null|[localservices]| 3D Printing|
 abruzzese| null| [IT]| [italian]| Abruzzese|
 absinthebars| null| [CZ]| [bars]| Absinthe Bars|
 acaibowls|[CL, PL, IT, TR, ...| null| [food]| Acai Bowls|
 accessories| null| null| [fashion]| Accessories|
 accountants| null| null| [professional]| Accountants|
 acnetreatment| null| null| [beautysvc]| Acne Treatment|
 active| null| null| []| Active Life|
 acupuncture| null| null| [health]| Acupuncture|
 addictionmedicine| [JP]| null| [physicians]| Addiction Medicine|
 adoptionservices| null| null|[localservices]| Adoption Services|
 adult| null| null| [shopping]| Adult|
 adultedu| null| null| [education]| Adult Education|
adultentertainment| null| null| [nightlife]|Adult Entertainment|
 advertising| null| null| [professional]| Advertising|
 aerialfitness| null| null| [fitness]| Aerial Fitness|
 aerialtours| null| null| [tours]| Aerial Tours|
 aestheticians| [AR, MX, CL]| null| [medicalspa]| Aestheticians|
 afghani| [TR, MX]| null| [restaurants]| Afghan|
 african| [TR]| null| [restaurants]| African|
+------------------+--------------------+-----------------+---------------+-------------------+
only showing top 20 rows

root
-- alias: string (nullable = true)
-- country_blacklist: array (nullable = true)
 |-- element: string (containsNull = true)
-- country_whitelist: array (nullable = true)
 |-- element: string (containsNull = true)
-- parents: array (nullable = true)
 |-- element: string (containsNull = true)
-- title: string (nullable = true)

#####Then we need to get those records from the category dataset where the parents field will show 'no' value. This means that they are at the top-level of the hierarchy.

In [59]:
import pyspark.sql.functions as f

df_categories_toplevelrecords=df_categories.filter(f.size(df_categories.parents)== 0)
print("no of records:", df_categories_toplevelrecords.count())
df_categories_toplevelrecords.show(30, truncate=False)


no of records: 22
+------------------+-----------------+-----------------+-------+----------------------------+
alias |country_blacklist|country_whitelist|parents|title |
+------------------+-----------------+-----------------+-------+----------------------------+
active |null |null |[] |Active Life |
arts |null |null |[] |Arts & Entertainment |
auto |null |null |[] |Automotive |
beautysvc |null |null |[] |Beauty & Spas |
bicycles |null |[CZ, PT, DK, PL] |[] |Bicycles |
education |null |null |[] |Education |
eventservices |null |null |[] |Event Planning & Services |
financialservices |null |null |[] |Financial Services |
food |null |null |[] |Food |
health |null |null |[] |Health & Medical |
homeservices |null |null |[] |Home Services |
hotelstravel |null |null |[] |Hotels & Travel |
localflavor |null |null |[] |Local Flavor |
localservices |null |null |[] |Local Services |
massmedia |null |null |[] |Mass Media |
nightlife |null |null |[] |Nightlife |
pets |null |null |[] |Pets |
professional |null |null |[] |Professional Services |
publicservicesgovt|null |null |[] |Public Services & Government|
religiousorgs |null |null |[] |Religious Organizations |
restaurants |null |null |[] |Restaurants |
shopping |null |null |[] |Shopping |
+------------------+-----------------+-----------------+-------+----------------------------+

#####Then, we would need to match up the records in the business dataset to the top-level categories. This can be done using the 'categories' field from the business dataframe and the 'title' field from the categories dataframe.

#####The categories field in the business dataset will also be used and will be compared to the 5 selected top-level categories from the categories dataset at a later step. This would be done by taking the selected categories and  checking if it is *in* the comma-separated list of values in the 'categories' field in the business DataFrame

#####The categories field in the business dataframe is currently in the form of strings. We will use Spark's split function which will allow us to split a string on commas, and the split function returns an array. Further, we could use the 'array_contains' function in Spark. One issue we will run into when splitting on the comma is that there might be a blank space before or after the commas. Hence the expression '\s*,\s*' is used as shown below.

In [61]:
df_business_tocategorylist = df_business_selected.select("business_id","state",f.split(df_business_selected.categories,'\s*,\s*').\
                                      alias("category"))
print("no of records:",df_business_tocategorylist.count())

df_business_tocategorylist.show(30,truncate=False)

no of records: 188593
+----------------------+-----+-----------------------------------------------------------------------------------------------------------------------+
business_id |state|category |
+----------------------+-----+-----------------------------------------------------------------------------------------------------------------------+
Apn5Q_b6Nz61Tq4XzPdf9A|AB |[Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel] |
AjEbIBw6ZFfln7ePHha9PA|NV |[Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services]|
O8S5hYJ1SMc8fA4QBtVujA|QC |[Breakfast & Brunch, Restaurants, French, Sandwiches, Cafes] |
bFzdJJ3wp3PZssNEsyU23g|AZ |[Insurance, Financial Services] |
8USyCYqpScwiNEb58Bt6CA|AB |[Home & Garden, Nurseries & Gardening, Shopping, Local Services, Automotive, Electronics Repair] |
45bWSZtniwPRiqlivpS8Og|AZ |[Coffee & Tea, Food] |
9A2quhZLyWk0akUetBd8hQ|ON |[Food, Bakeries] |
6OuOZAok8ikONMS_T3EzXg|ON |[Restaurants, Thai] |
8-NRKkPY1UiFXW20WXKiXg|AZ |[Mexican, Restaurants] |
UTm5QZThPQlT35mkAcGOjg|PA |[Flowers & Gifts, Gift Shops, Shopping] |
KapTdGyGs7RK0c68Z6hhhg|ON |[Restaurants, Japanese] |
tZnSodhPwNr4bzrwJ1CSbw|ON |[Cajun/Creole, Southern, Restaurants] |
_c3ixq9jYKxhLUB0czi0ug|AZ |[Bars, Sports Bars, Dive Bars, Burgers, Nightlife, Sandwiches, Restaurants] |
bBUMib8l6Me1ZB1_Qkezkg|QC |[Restaurants, Pakistani, Indian, Middle Eastern] |
o_zDURWzgt5yM5HW_xGJsg|AB |[Beauty & Spas, Barbers] |
IbTDLI02Sq50C7B-EWK1sg|AB |[Delis, Restaurants, Sandwiches] |
r89Re4FNgVWHgBfjCVZyVw|QC |[Nightlife, Bars, American (Traditional), Tapas/Small Plates, Poutineries, Supper Clubs, Restaurants, Tapas Bars] |
hTzcHtk4-0QJnFUbkKpd5Q|NV |[Shopping, Fashion, Department Stores] |
UwIpS9UKsPiKAv1fiEYhqg|NV |[Financial Services, Check Cashing/Pay-day Loans, Title Loans] |
vJIuDBdu01vCA8y1fwR1OQ|NV |[American (Traditional), Food, Bakeries, Restaurants] |
YZCHr68c5aEVHz0bkq9K2g|NV |[Home Services, Masonry/Concrete, Professional Services, Contractors] |
gJ5xSt6147gkcZ9Es0WxlA|OH |[Fast Food, Burgers, Restaurants] |
3ByGQOVgds2YEu6kzl-XEQ|AZ |[Nightlife, Bars, American (Traditional), Sports Bars, Restaurants] |
5J3b7j3Fzo9ISjChmoUoUA|ON |[Food, Bakeries, Coffee & Tea] |
CeuTRtwsq6w5rztGOyNMPg|AZ |[Mexican, Restaurants] |
gsoSAb9t7Ar7Ww6O_3GuOg|IL |[Mexican, Restaurants] |
guRyrKMBDkcHExGzwxSfvg|AZ |[Home Services, Home Automation, Security Systems] |
PMDlKLd0Mxj0ngCpuUmE5Q|ON |[Restaurants, Food, Canadian (New), Coffee & Tea] |
65DAbjsiws2_T600pNpWFQ|ON |[Optometrists, Shopping, Eyewear & Opticians, Health & Medical] |
6YC6CsXRrmPv_iwfvc9onA|OH |[Pizza, Restaurants] |
+----------------------+-----+-----------------------------------------------------------------------------------------------------------------------+
only showing top 30 rows

####From the top-level categories list, we have selected 5 top-level categories based on the Yelp 10K file. The 5 top-level categories are Restaurants,  Shopping, Nightlife, Home Services, Arts & Entertainment.

Page 6 of the 'YelpInc 10K' file has a chart that provides information on the reviews contributed to Yelp's platform. The review contributions cover a wide set of local business categories,including restaurants, shopping, beauty and fitness, arts, entertainment and events, home and local services, health, nightlife, travel and hotel, auto and other categories.

The chart provided below provides the breakdown by industry of local businesses that have received reviews on Yelp's platform and the breakdown by industry of reviews contributed to their platform through December 31, 2016. The decision to select the 5 top-level categories have been based on this chart. 

Another point to be noted is that, the chart shows 'Home & Local Services' as one category but the categories file has 'Home Services' and 'Local Services' as 2 separate top-level categories. We have decided to go with 'Home Services' from the categories dataset provided but are unsure on the impact it will create. In the same way, the chart considers 'Arts, Entertainment and Events' as a single category but the categories dataframe shows 'Arts & Entertainment' separate to 'Event Planning & Services'. Similarly, we have decided to chose 'Arts & Entertainment' and are not sure on the impact it would create on our analysis. This likely could be a scope that would need to be considered as part of another project.

In the next steps, we check to see if the top levle categories are within the category list in the business dataframe

In [63]:
displayHTML(showimage("/dbfs/FileStore/tables/Yelp10KChart_onTLCategories.png"))

#####The 5 selected top-level categories have been shown as 'is_restaurants', 'is_shopping', 'is_nighlife', 'is_home_services' and 'is_arts_entertainment'. This allows the categories to be shown as 'true', and we only need such records for the data analysis. Using sql joins could have been complicated and hence we decided to opt for this data wrangling method.

In [65]:
df_business_categorylist_5selectedcategorylist = df_business_tocategorylist.select("business_id","state","category",f.array_contains(df_business_tocategorylist.category, 'Restaurants').alias('is_restaurants'),f.array_contains(df_business_tocategorylist.category, 'Shopping').alias('is_shopping'), f.array_contains(df_business_tocategorylist.category, 'Nightlife').alias('is_nightlife'),f.array_contains(df_business_tocategorylist.category, 'Home Services').alias('is_home_services'),f.array_contains(df_business_tocategorylist.category, 'Arts & Entertainment').alias('is_arts_entertainment'))
print("no. of records:",df_business_categorylist_5selectedcategorylist.count())
df_business_categorylist_5selectedcategorylist.show()

no. of records: 188593
+--------------------+-----+--------------------+--------------+-----------+------------+----------------+---------------------+
 business_id|state| category|is_restaurants|is_shopping|is_nightlife|is_home_services|is_arts_entertainment|
+--------------------+-----+--------------------+--------------+-----------+------------+----------------+---------------------+
Apn5Q_b6Nz61Tq4Xz...| AB|[Tours, Breweries...| true| false| false| false| false|
AjEbIBw6ZFfln7ePH...| NV|[Chicken Wings, B...| true| false| false| false| false|
O8S5hYJ1SMc8fA4QB...| QC|[Breakfast & Brun...| true| false| false| false| false|
bFzdJJ3wp3PZssNEs...| AZ|[Insurance, Finan...| false| false| false| false| false|
8USyCYqpScwiNEb58...| AB|[Home & Garden, N...| false| true| false| false| false|
45bWSZtniwPRiqliv...| AZ|[Coffee & Tea, Food]| false| false| false| false| false|
9A2quhZLyWk0akUet...| ON| [Food, Bakeries]| false| false| false| false| false|
6OuOZAok8ikONMS_T...| ON| [Restaurants, Thai]| true| false| false| false| false|
8-NRKkPY1UiFXW20W...| AZ|[Mexican, Restaur...| true| false| false| false| false|
UTm5QZThPQlT35mkA...| PA|[Flowers & Gifts,...| false| true| false| false| false|
KapTdGyGs7RK0c68Z...| ON|[Restaurants, Jap...| true| false| false| false| false|
tZnSodhPwNr4bzrwJ...| ON|[Cajun/Creole, So...| true| false| false| false| false|
_c3ixq9jYKxhLUB0c...| AZ|[Bars, Sports Bar...| true| false| true| false| false|
bBUMib8l6Me1ZB1_Q...| QC|[Restaurants, Pak...| true| false| false| false| false|
o_zDURWzgt5yM5HW_...| AB|[Beauty & Spas, B...| false| false| false| false| false|
IbTDLI02Sq50C7B-E...| AB|[Delis, Restauran...| true| false| false| false| false|
r89Re4FNgVWHgBfjC...| QC|[Nightlife, Bars,...| true| false| true| false| false|
hTzcHtk4-0QJnFUbk...| NV|[Shopping, Fashio...| false| true| false| false| false|
UwIpS9UKsPiKAv1fi...| NV|[Financial Servic...| false| false| false| false| false|
vJIuDBdu01vCA8y1f...| NV|[American (Tradit...| true| false| false| false| false|
+--------------------+-----+--------------------+--------------+-----------+------------+----------------+---------------------+
only showing top 20 rows

The view, 'view_business_5selectedcategorylist' is created so that this could be joined with the 'view_review_compare_toavgOfbusiness_original' to get a dataframe which would be used in tableau to answer the question asked.  Similarly the 'view_business_5selectedcategorylist' could also be joined to 'view_review_compare_toavgOfbusiness', to export results to tableau and show only for those businesses with review count above 10.

In [67]:
#The view has been named as 'view_business_5selectedcategorylist' since it gives records of businesses and the records associated to the selected 5 top-level categories
df_business_categorylist_5selectedcategorylist.createOrReplaceTempView("view_business_5selectedcategorylist")

##### Since we have got the businesses pertaining to our selected categories, we would need to join them to our review dataset in order to observe any patterns in the ratings based on category. Here we are finding out the results using the view created from the review dataframe prior to filtering. We do this to find out about the differences.

In [69]:
df_review_business_selectedcategories_original=spark.sql("""
SELECT vbcl.business_id, vrcb.review_id, vrcb.stars, vrcb.avgstars, vrcb.diff_avgstars, vrcb.month, vrcb.year, vbcl.state, vbcl.is_restaurants, vbcl.is_shopping, vbcl.is_nightlife, vbcl.is_home_services, vbcl.is_arts_entertainment
FROM view_review_compare_toavgOfbusiness_original vrcb
JOIN view_business_5selectedcategorylist vbcl ON vrcb.business_id = vbcl.business_id
""")
print("no. of records:",df_review_business_selectedcategories_original.count())
df_review_business_selectedcategories_original.show(50)

no. of records: 5996996
+--------------------+--------------------+-----+--------+-------------+-----+----+-----+--------------+-----------+------------+----------------+---------------------+
 business_id| review_id|stars|avgstars|diff_avgstars|month|year|state|is_restaurants|is_shopping|is_nightlife|is_home_services|is_arts_entertainment|
+--------------------+--------------------+-----+--------+-------------+-----+----+-----+--------------+-----------+------------+----------------+---------------------+
--9e1ONYQuAa-CB_R...|lgpSS6UsKYIvnQaw8...| 5| 4.1028| 0.8972| 4|2005| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|B45SjrSgVXDw2EPlF...| 5| 4.1028| 0.8972| 1|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|rIwd3sRWVrN7d0vxd...| 5| 4.1028| 0.8972| 1|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|aQQIcQfFYawLYU85o...| 5| 4.1028| 0.8972| 3|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|-7O72HqM4eJF0NMCg...| 5| 4.1028| 0.8972| 3|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|Y9mls1kWNRsHYURCt...| 4| 4.1028| -0.1028| 5|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|d3-sC4eUvIdDzz6Kg...| 4| 4.1028| -0.1028| 7|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|y3EBG-1zykvZblLdS...| 4| 4.1028| -0.1028| 7|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|tCpIO0BO1f916-Ch9...| 5| 4.1028| 0.8972| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|bHMxqjTDchcYo8pqw...| 4| 4.1028| -0.1028| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|8Lkdb_TyJDn8knJX0...| 4| 4.1028| -0.1028| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|pg2KJCg1stKrexSzg...| 5| 4.1028| 0.8972| 11|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|He1MijYelqA7uZxH0...| 5| 4.1028| 0.8972| 12|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|WtIWqoHjAruu__QYt...| 4| 4.1028| -0.1028| 1|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|tYIxOLwcZfnC5Wrht...| 3| 4.1028| -1.1028| 2|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|IsFiM2sCdnkYnQN1K...| 5| 4.1028| 0.8972| 3|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|miW64R3yPbSWwCJtj...| 4| 4.1028| -0.1028| 3|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|z6RWTqn5dZYPSQxx1...| 3| 4.1028| -1.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|tZIL8bm90mjE7zgtd...| 5| 4.1028| 0.8972| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|hwmEJHtumKQOHqe85...| 3| 4.1028| -1.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|-cnWnD1w8gHyqr62N...| 2| 4.1028| -2.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|T4Q7lLvwhN-VuHGUi...| 3| 4.1028| -1.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|dasxQe5zVeI-mX09O...| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|4P3cSgHjWB41_fgbR...| 3| 4.1028| -1.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|I1SZsm5u53gnjDWL0...| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|zBiEZwq2M95CVtI3f...| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|k4Jo3PHT_NjCP_-oR...| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|cFScDKl6_cTT8X_Hv...| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|qIjeZVcHCN8CUdkkU...| 5| 4.1028| 0.8972| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|6gP4H1CD7kjCaQLwu...| 3| 4.1028| -1.1028| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|fQFMt6aAABIt9Ntyd...| 5| 4.1028| 0.8972| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...|k0qS-yOEa0Qau4pIW...| 3| 4.1028| -1.1028| 7|2007| NV| true| false| false| false| false|
--9e1ONYQu

#####As explained previously, this step would give us results similar to the step above but for businesses with review count greater than 10.

In [71]:
df_review_business_selectedcategories=spark.sql("""
SELECT vbcl.business_id, vrcb.reviewcount, vrcb.stars, vrcb.avgstars, vrcb.diff_avgstars, vrcb.month, vrcb.year, vbcl.state, vbcl.is_restaurants, vbcl.is_shopping, vbcl.is_nightlife, vbcl.is_home_services, vbcl.is_arts_entertainment
FROM view_review_compare_toavgOfbusiness vrcb
JOIN view_business_5selectedcategorylist vbcl ON vrcb.business_id = vbcl.business_id
""")
print("no. of records:",df_review_business_selectedcategories.count())
df_review_business_selectedcategories.show(50)

no. of records: 5457875
+--------------------+-----------+-----+--------+-------------+-----+----+-----+--------------+-----------+------------+----------------+---------------------+
 business_id|reviewcount|stars|avgstars|diff_avgstars|month|year|state|is_restaurants|is_shopping|is_nightlife|is_home_services|is_arts_entertainment|
+--------------------+-----------+-----+--------+-------------+-----+----+-----+--------------+-----------+------------+----------------+---------------------+
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 4|2005| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 1|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 1|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 5|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 7|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 7|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 9|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 11|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 12|2006| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 1|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 2|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 3|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 3|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 2| 4.1028| -2.1028| 4|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 6|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 3| 4.1028| -1.1028| 7|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 8|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 0.8972| 8|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 8|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 8|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 4| 4.1028| -0.1028| 8|2007| NV| true| false| false| false| false|
--9e1ONYQuAa-CB_R...| 1546| 5| 4.1028| 

#####The records are downloaded into a '.tsv' file and then imported into tableau for data visualization. The steps to create a '.tsv' file and move it within the DBFS is shown below. The steps are similar to what has been explained previously when the results from the wrangled review dataset were moved into the tables folder within the DBFS.

In [73]:
#df_review_business_selectedcategories.repartition(1).write.option("header","true").option("sep","\t").option("mode","overwrite").csv("/FileStore/tables/review_business_selectedcategories_filtered")

In [74]:
#dbutils.fs.mv("/FileStore/tables/review_business_selectedcategories_filtered/part-00000-tid-1440271551077644682-0ba46108-3c99-4298-8d91-e290769d8964-3822-c000.csv", "/FileStore/tables/review_business_selectedcategories_filtered.tsv")

In [75]:
#dbutils.fs.rm("/FileStore/tables/review_business_selectedcategories_filtered", recurse=True)

In [76]:
#Paste the below link in the browser to retrieve the review_business_selectedcategories_filtered.tsv file for analysing the categories and state part of the question
#https://community.cloud.databricks.com/files/tables/review_business_selectedcategories_filtered.tsv?o=5722899434425902#notebook/

#####The resulting records obtained by joining the results from the 'data-wrangled' review dataset and by using the selected categories. These are then imported into tableau for Data visualization to further help us answer our question.

####Data visualization based on the 5 selected top-level category

#####Comparison over months for 2007 vs 2017. Users rate businesses higher than the average, but no monthly pattern observed.

In [80]:
displayHTML(showimage("/dbfs/FileStore/tables/ReviewCountAbv10_2007.png"))

In [81]:
displayHTML(showimage("/dbfs/FileStore/tables/ReviewCountAbv10_2017.png"))

#####When comparing the businesses for Restaurants acorss the various years, most users rate higher than the average but no observable trend

In [83]:
displayHTML(showimage("/dbfs/FileStore/tables/ReviewCountAbv10_inYears.png"))

#####For Shopping, when comparison over months for 2007 vs 2017, we find that users rate businesses slightly higher than the average, but no monthly trend observed

In [85]:
displayHTML(showimage("/dbfs/FileStore/tables/Shopping_2007.png"))

In [86]:
displayHTML(showimage("/dbfs/FileStore/tables/Shopping_2017.png"))

#####For shopping when taken across years, users seem to rate the businesses slightly higher than the average

In [88]:
displayHTML(showimage("/dbfs/FileStore/tables/Shopping_years.png"))

##### For Nightlife, Comparison over months for 2007 vs 2017. Users rate businesses higher than the average, but no monthly pattern observed as compared to the initial years where the median was higher for certain months

In [90]:
displayHTML(showimage("/dbfs/FileStore/tables/Nightlife_2007.png"))

In [91]:
displayHTML(showimage("/dbfs/FileStore/tables/Nightlife_2017.png"))

##### For Nightlife, the user rating is higher than the average. 2016 saw some businesses rated comparatively higher than the other years

In [93]:
displayHTML(showimage("/dbfs/FileStore/tables/Nightlife_years.png"))

##### For HomeServices, Comparison over months for 2007 vs 2017. Users hardly rated much home services businesses in the early years of Yelp. Of late, they rate it higher than the average, although the summers have a higher dispersion of ratings than the other months

In [95]:
displayHTML(showimage("/dbfs/FileStore/tables/HomeServices_2007.png"))

In [96]:
displayHTML(showimage("/dbfs/FileStore/tables/HomeServices_2017.png"))

##### Home Services over the years. The initial four years show random variation; the box seems to be getting narrower in the last 5 years, with user ratings getting closer to the average each year

In [98]:
displayHTML(showimage("/dbfs/FileStore/tables/HomeServices_Year.png"))

##### For Arts & Entertainment, Ratings fluctuated both up and down during the initial years but evened our in the recent years. Users rate businesses higher than the average but no trend is observed over the months

In [100]:
displayHTML(showimage("/dbfs/FileStore/tables/A_E_2007-68a3e.png"))

In [101]:
displayHTML(showimage("/dbfs/FileStore/tables/A_E_2007-68a3e.png"))

##### Arts & Entertainment over the years; users rate businesses higher than the average; 2015 saw a higher maximum rating than other years

In [103]:
displayHTML(showimage("/dbfs/FileStore/tables/A_E_2007-68a3e.png"))

####Data visualization based on the geographical location

##### Alberta (AB): the median holds around the same value for each month but the max and min ratings are low as compared to other states. Arizona (AZ): a large number of businesses either have a lower or higher rating than the average, as compared to the other states. Illinois (IL): the spread for May and September is narrow; February and April have the widest variance for ratings. North Carolina (NC): for some reasons, we observe a lot of outliers for this state, suggesting errors (specially before the 1st quartile). Nevada (NV): a large number of outliers; this needs to be looked into, to observe if a particular driver is behind it. Ohio (OH): the dispersion is narrower in June, October and December, as compared to other months. Ontario (ON): higher than average. Pennsylvania (PA): no observable monthly variance in ratings which are all higher than the average. South Carolina (SC): of all the states, SC has the least difference from the average ratings. And they seem to variate each month too (although higher than the average). Wisconsin (WI) does not show much variations. 

Here we would only be showing for a few graphs.

#####South Carolina (SC): of all the states, SC has the least difference from the average ratings. And they seem to variate each month too (although higher than the average)

In [107]:
displayHTML(showimage("/dbfs/FileStore/tables/State_SC.png"))

#####We can see here how there is not much difference in the user ratings from the average ratings in the initial years (mostly because of lack of data). However this starts to change from 2010 onwards. Arizona and Nevada have the biggest difference of the lot

In [109]:
displayHTML(showimage("/dbfs/FileStore/tables/State_Year.png"))

######From the above analysis, we infer that Users rate the businesses mostly higher than the average but there is no monthly trend to it.

######Our project allows us to answer if the users rate businesses higher or lower than the business category, but it doesn’t tell by how much or what percentage of users rate it higher/lower. Yelp can build onto it

#####Using the project’s findings, the Yelp team can identify the businesses who fared lower than the average ratings. This could be either be consumer related or based on business functions across different states where seasonal fluctuations were observed